# 📈 KOSPI 시총 상위 10종목 데이터 자동화 노트북

Recent 1년간 **KOSPI 상위 10개 종목**의 주가 데이터를 수집하고:

- 거래량 0 제거 & 결측값 보간
- 일별 수익률 계산
- 거래량 상위 5일 추출

을 수행한 뒤 **Google 스프레드시트**에 업로드합니다.


## 🔐 Google 스프레드시트 API 인증 설정

1. **Google Cloud Console**에서 프로젝트 생성  
2. **APIs & Services → Enabled APIs & services**에서  
   - Google Sheets API  
   - Google Drive API  
   를 각각 **Enable**  
3. **APIs & Services → Credentials → Create credentials → Service account** 생성  
   - 역할: Editor  
4. 서비스 계정 생성 후 **Keys → Add key → JSON**  
   - `credentials.json` 다운로드  
5. 스프레드시트에서 **공유** → 서비스 계정 이메일(예: `xxx@yyy.iam.gserviceaccount.com`)에 편집 권한 부여  
6. 이 노트북과 동일 폴더에 `credentials.json` 배치  


In [ ]:
# 필요한 패키지 설치 및 임포트
# !pip install yfinance pandas numpy gspread gspread_dataframe

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

import gspread
from gspread_dataframe import set_with_dataframe


In [ ]:
# KOSPI 시총 상위 10개 종목 티커 (Yahoo Finance)
tickers = {
    'SamsungElec': '005930.KS',
    'SKhynix': '000660.KS',
    'HyundaiMotor': '005380.KS',
    'LGChem': '003550.KS',
    'SamsungSDI': '006400.KS',
    'POSCO': '005490.KS',
    'Naver': '035420.KS',
    'Kia': '000270.KS',
    'HyundaiMobis': '012330.KS',
    'LGEnergySol': '373220.KS'
}

# 최근 1년 데이터 기간 설정
end = datetime.today()
start = end - timedelta(days=365)

# 다운로드
raw = yf.download(
    tickers=list(tickers.values()),
    start=start, end=end,
    group_by='ticker',
    auto_adjust=False
)

# tidy to MultiIndex DataFrame
frames = []
for name, symbol in tickers.items():
    df = raw[symbol].copy()
    df['Ticker'] = name
    frames.append(df)

price_df = pd.concat(frames)
price_df.index.names = ['Ticker', 'Date']
price_df.head()


In [ ]:
# 1) 거래량 0 제거, 2) 결측치는 직전 값으로 대체
clean = (
    price_df[price_df['Volume'] != 0]
    .groupby(level=0)
    .apply(lambda x: x.ffill())
)

# 일일 수익률 계산
clean['Return'] = clean['Close'].pct_change()

# 거래량 상위 5일 추출
top5 = (
    clean.reset_index()
    .sort_values(['Ticker', 'Volume'], ascending=[True, False])
    .groupby('Ticker')
    .head(5)
)

print("거래량 상위 5일:")
display(top5[['Ticker', 'Date', 'Volume']])


In [ ]:
# Google Sheets 업로드
SHEET_NAME = 'KOSPI_TOP10_1Y'
CREDS_FILE = 'credentials.json'

gc = gspread.service_account(filename=CREDS_FILE)
try:
    sh = gc.open(SHEET_NAME)
except gspread.SpreadsheetNotFound:
    sh = gc.create(SHEET_NAME)
    # 필요 시 실제 사용자 이메일로 공유: sh.share('you@example.com', perm_type='user', role='writer')

# 전체 데이터 시트
ws_all = sh.sheet1
set_with_dataframe(ws_all, clean.reset_index())

# 거래량 TOP5 시트
try:
    ws_top = sh.worksheet('TOP5_VOLUME')
except gspread.WorksheetNotFound:
    ws_top = sh.add_worksheet(title='TOP5_VOLUME', rows='100', cols='20')
set_with_dataframe(ws_top, top5)

print("Google 스프레드 시트 업로드 완료!")


## 🖥️ GUI 실행 파일 제작

자동화 스크립트를 **GUI**로 실행하려면:

```bash
pip install pyinstaller
```

`app.py` 파일 생성 후:

```bash
pyinstaller --onefile --noconsole app.py
```

- `--onefile`: 단일 실행 파일 생성  
- `--noconsole`: 콘솔 창 숨김  

`dist/app.exe` 실행 시 버튼 클릭만으로 작업이 수행됩니다.
